In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from brainviewer import NapariBrainViewer

## Generating very very bad fake neural activity

In [3]:
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

In [4]:
# parameters of the simulation
space_size = 100
s_xyz = np.array([space_size,space_size,space_size])

n_regions = 15
n_times = 3000
n_neurons = 10000

In [5]:
# creating regions

## spatial
com_regions = np.random.uniform(low=0*s_xyz, high=s_xyz, size=(n_regions, 3)) # centers of masse


## temporal
p_join_regions = np.random.choice( # probabilities of 2 regions activating simultaniously
    [0,1,0.1,0.9,0],
    p = [0.3,0.3,0.175,0.175,0.05],
    size=(n_regions, n_regions),
)
p_join_regions = ( p_join_regions + p_join_regions.conj().T )/2

region_acts = np.zeros((n_times, n_regions)) # random telegraph signal for each region with p_join contraint
p = 0.001
ref_j = 0
for i in range(1,n_times):
    if np.random.rand() < p*2:
        ref_j = np.random.randint(0,n_regions)
    if np.random.rand() < p:
        region_acts[i:, ref_j] = 1-region_acts[i-1, ref_j]
    for j in range(n_regions):
        if j == ref_j:
            continue
        if np.random.rand() < p_join_regions[ref_j,j]:
            region_acts[i:,j] = region_acts[i:,ref_j]
        else:
            region_acts[i:,j] = 1-region_acts[i:,ref_j]

In [6]:
# creating neurons

## spatial
com_neurons = np.random.uniform(low=0*s_xyz, high=s_xyz, size=(n_neurons, 3)) # centers of masse


## participation of neurons in each region
dists = distance_matrix(com_neurons, com_regions)
def norm_tanh(x, x0=20, width=10):
    xmin, xmax = x.min(), x.max()
    x = (x-xmin) / (xmax - xmin)
    x0 = (x0-xmin) / (xmax - xmin)
    width = (width-xmin) / (xmax - xmin)
    y = (1 - np.tanh((x-x0)/width*np.pi))/2
    return y/y.max()
participation_factor = norm_tanh(dists)

## temporal
neuron_tendencies = region_acts@participation_factor.T
neuron_spikes = np.random.poisson(neuron_tendencies*5)

## Opening the brain viewer

In [7]:
nbv = NapariBrainViewer()

## Scatter Neurons

In [8]:
layer_com = nbv.points(com_neurons, blending="translucent_no_depth", size=2)

## Scatter neurons with single frame activity

In [9]:
from brainviewer import cm_inferno_alpha

In [10]:
layer_activity = nbv.points(
    com_neurons, neuron_spikes[50,:], 
    cmap=cm_inferno_alpha, crange=(0,+5), 
    blending="translucent_no_depth",
    size=2,
)

In [11]:
layer_activity.features = {"val":neuron_spikes[63,:]}

In [12]:
from qtpy.QtWidgets import QSlider, QLabel, QHBoxLayout, QWidget, QPushButton, QDoubleSpinBox
from qtpy.QtCore import Qt, QTimer

In [13]:
def slider_callback(i):
    layer_activity.features = {"val":neuron_spikes[i,:]}

class TimeSlider(QWidget):
    def __init__(self, n_times, callback):
        super().__init__()
        self.n = n_times
        self.callback = callback

        # Timer for automatic slider movement
        self.timer = QTimer(self)
        self.timer.timeout.connect(self._advance_slider)

        self.setLayout(QHBoxLayout())
        self.layout().addWidget(QLabel("Time Slider"))

        self.playbutton = QPushButton("Play")
        self.playbutton.setCheckable(True)
        self.playbutton.clicked.connect(self._toogle_play)
        self.playbutton.setStyleSheet("background-color : black")
        self.layout().addWidget(self.playbutton)

        self.fps_control = QDoubleSpinBox()
        self.fps_control.setRange(0.1, 100.0)  # Set FPS range from 0.1 to 100
        self.fps_control.setDecimals(1)  # Allow one decimal place
        self.fps_control.setValue(5.0)  # Default FPS value
        self.fps_control.setSuffix(" FPS")  # Add suffix to indicate FPS
        self.fps_control.valueChanged.connect(self._update_timer_interval)  # Update timer when FPS changes
        self.layout().addWidget(self.fps_control)

        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(0)
        self.slider.setMaximum(self.n-1)
        self.slider.setValue(self.n//2)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(1)
        self.layout().addWidget(self.slider)
        self.slider.valueChanged.connect(self._valuechanged)
    
        self.timelabel = QLabel(self._time_label(self.n//2))
        self.layout().addWidget(self.timelabel)

    def _time_label(self, i):
        return f" t={i} "

    def _valuechanged(self):
        i = self.slider.value()
        self.timelabel.setText(self._time_label(i))
        self.callback(i)

    def _toogle_play(self):
        if self.playbutton.isChecked():
            self.playbutton.setStyleSheet("background-color : darkgrey")
            self.playbutton.setText("Pause")
            self._update_timer_interval()
            self.timer.start()
        else:
            self.playbutton.setStyleSheet("background-color : black")
            self.playbutton.setText("Play")
            self.timer.stop()

    def _advance_slider(self):
        current_value = self.slider.value()
        if current_value < self.n - 1:
            self.slider.setValue(current_value + 1)
        else:
            self.slider.setValue(0)

    def _update_timer_interval(self):
        fps = self.fps_control.value()
        interval = 1000 / fps  # Convert FPS to milliseconds (1000 ms per second)
        self.timer.setInterval(int(interval))

In [14]:
widget = TimeSlider(3000, slider_callback)

In [15]:
a = nbv._v.window.add_dock_widget(widget, area="bottom")

In [16]:
widget.close()
a.close()

True

In [20]:
if (nbv._v.dims.ndim > 3) and ("t" in nbv._v.dims.axis_labels):
    print("not ok")
else:
    print("ok")

ok
